In [0]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 63kB/s 
     |████████████████████████████████| 204kB 52.5MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=7705ce92694f2f4773a9d75f76fdda5591310da17ca49e7b077703c143a6ce06
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
!java -version
!sudo update-alternatives --config java
!java -version

openjdk version "11.0.4" 2019-07-16
OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3)
OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (b

In [0]:
import numpy as np
import pandas as pd

from operator import add
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler
from pyspark.ml.feature import Word2Vec, Word2VecModel
from pyspark.ml.feature import IDF

from pyspark.ml import Pipeline, PipelineModel


from pyspark.sql.functions import *

from pyspark.sql.types import *

import folium
import html
import pickle
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.session import SparkSession

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
with open('gdrive/My Drive/Yelp Data/dfRestaurants.pickle', 'rb') as f:
    df_business = pickle.load(f)

In [0]:
sc = SparkContext('local')
spark = SparkSession(sc)

In [0]:
df_business_spark = spark.createDataFrame(df_business)

In [0]:
df_business_spark.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_count: long (nullable = true)
 |-- is_open: long (nullable = true)
 |-- categories: string (nullable = true)



In [0]:
df_business_spark.show(5)

+--------------------+--------------------+------------------+--------------------+-------+-----+-----------+-----------------+------------------+-----+------------+-------+--------------------+
|         business_id|                name|      neighborhood|             address|   city|state|postal_code|         latitude|         longitude|stars|review_count|is_open|          categories|
+--------------------+--------------------+------------------+--------------------+-------+-----+-----------+-----------------+------------------+-----+------------+-------+--------------------+
|l09JfMeQ6ynYs5MCJ...|    "Alize Catering"|Yonge and Eglinton|     "2459 Yonge St"|Toronto|   ON|    M4P 2H6|       43.7113993|       -79.3993388|  3.0|          12|      0|Italian;French;Re...|
|1K4qrnfyzKzGgJPBE...|"Chula Taberna Me...|       Leslieville|"1058 Gerrard Str...|Toronto|   ON|    M4M 3A6|43.66925620000001|       -79.3359022|  3.5|          39|      1|Tiki Bars;Nightli...|
|dTWfATVrBfKj7Vdn0...|   

In [0]:
with open('gdrive/My Drive/Yelp Data/reviews_df.pickle', 'rb') as f:
    df_reviews = pickle.load(f)

In [0]:
df_reviews = df_reviews.sample(20000)

In [0]:
user_concatreviews = df_reviews[['user_id', 'text']].groupby(['user_id'])['text'].apply(lambda x: ' '.join(x)).reset_index()

In [0]:
user_concatreviews

,user_id,text
0,---1lKK3aKOuomHnwAkAow,They host Delivering Happiness Inspire! and th...
1,--104qdWvE99vaoIsj9ZJQ,When you buy a couple of drinks and they taste...
2,--2vR0DIsmQ6WfcSzKWigw,Unser eher bescheidenes Motel war in der Nähe ...
3,--B1_68kNRRoT9k61Qlu3g,I love this salon. The staff is super knowledg...
4,-0S_XaK3Q_Mesal2Unta2w,"Ty fun have the best Pad Thai I ever have,and ..."
...,...,...
18651,zzDN7KhMfU5UPbTUSfYSyQ,John is the best... a very honest and reasonab...
18652,zzKA4xBV5sI5yBDo1XC0cw,THANK THE LORD FOR THE ABC STORE:\n\nAfter air...
18653,zzPvEodjvLPe-5NvV0U_cg,If you are shopping for an US def do not go to...
18654,zzUEa4doDSxQfzDUYkopJA,We have been to this restaurant about 4 times....


In [0]:
business_concatreviews = df_reviews[['business_id', 'text']].groupby(['business_id'])['text'].apply(lambda x: ' '.join(x)).reset_index()

In [0]:
business_concatreviews

,business_id,text
0,--9e1ONYQuAa-CB_Rrw7Tw,I did the 2012 Restaurant Week $30.12 lunch sp...
1,--KQsXc-clkO7oHRqGzSzg,I came here after Christmas with a friend who ...
2,--S62v0QgkqQaVUhFnNHrw,Went here this morning with my family as my pa...
3,--ab39IjZR_xUf81WyTyHg,"I really , really, really wish the Mesa Riverv..."
4,--cZ6Hhc9F7VkKXxHMVZSQ,My good was decent. The steak was good and my ...
...,...,...
14565,zx_xDVaVQlE3eDoFFENkow,I needed two iPhone 4s unlocked and an iPhone ...
14566,zy_NHTqtfSrfTGGPoqy4Mw,"delicious daily specials, cheerful and tasty, ..."
14567,zyf4e5p_4X2cNc_WPIiTbg,My daughter and I have been seeing Dr Lida Rao...
14568,zzTM2KyJkKomLDkl6rM4dQ,We've now had Mike out twice to fix some leaks...


In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stop = stopwords.words('english')
stemmer = SnowballStemmer("english")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
business_concatreviews['nostop'] = business_concatreviews['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
business_concatreviews['cleantext'] = business_concatreviews['nostop'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))
business_concatreviews = business_concatreviews.drop(columns = ['text', 'nostop'])

In [0]:
user_concatreviews['nostop'] = user_concatreviews['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
user_concatreviews['cleantext'] = user_concatreviews['nostop'].apply(lambda x: ' '.join([stemmer.stem(y) for y in x.split()]))
user_concatreviews = user_concatreviews.drop(columns = ['text', 'nostop'])

In [0]:
user_concatreviews

,user_id,cleantext
0,---1lKK3aKOuomHnwAkAow,they host deliv happi inspire! enough get 5 st...
1,--104qdWvE99vaoIsj9ZJQ,"when buy coupl drink tast like soda, know some..."
2,--2vR0DIsmQ6WfcSzKWigw,unser eher bescheiden motel war der nähe des r...
3,--B1_68kNRRoT9k61Qlu3g,i love salon. the staff super knowledg helpful...
4,-0S_XaK3Q_Mesal2Unta2w,"ty fun best pad thai i ever have,and curri goo..."
...,...,...
18651,zzDN7KhMfU5UPbTUSfYSyQ,john best... honest reason guy. i would recomm...
18652,zzKA4xBV5sI5yBDo1XC0cw,thank the lord for the abc store: after airpor...
18653,zzPvEodjvLPe-5NvV0U_cg,if shop us def go miracl progress!! come here!...
18654,zzUEa4doDSxQfzDUYkopJA,"we restaur 4 times. the food pretti good, staf..."


In [0]:
reviews_by_business_df = spark.createDataFrame(business_concatreviews)

In [0]:
regexTokenizer = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'cleantext', outputCol = 'token')
stopWordsRemover = StopWordsRemover(inputCol = 'token', outputCol = 'nostopwrd')
countVectorizer = CountVectorizer(inputCol="nostopwrd", outputCol="rawFeature")
iDF = IDF(inputCol="rawFeature", outputCol="idf_vec")
word2Vec = Word2Vec(vectorSize = 100, minCount = 5, inputCol = 'nostopwrd', outputCol = 'word_vec', seed=123)
vectorAssembler = VectorAssembler(inputCols=['idf_vec', 'word_vec'], outputCol='comb_vec')
pipeline = Pipeline(stages=[regexTokenizer, stopWordsRemover, countVectorizer, iDF, word2Vec, vectorAssembler])

# fit the model
pipeline_mdl = pipeline.fit(reviews_by_business_df)

#save the pipeline model
pipeline_mdl.write().overwrite().save('pipe_txt')

In [0]:
pipeline_mdl = PipelineModel.load('pipe_txt')


In [0]:
reviews_by_business_trf_df = pipeline_mdl.transform(reviews_by_business_df)


In [0]:
reviews_by_business_trf_df.select( 'cleantext', 'nostopwrd', 'idf_vec', 'word_vec', 'comb_vec').show(10)


+--------------------+--------------------+--------------------+--------------------+--------------------+
|           cleantext|           nostopwrd|             idf_vec|            word_vec|            comb_vec|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|i 2012 restaur we...|[2012, restaur, w...|(38670,[1,2,3,4,5...|[0.00620139042982...|(38770,[1,2,3,4,5...|
|i came christma f...|[came, christma, ...|(38670,[2,5,10,13...|[-0.0354065453426...|(38770,[2,5,10,13...|
|went morn famili ...|[went, morn, fami...|(38670,[0,1,6,7,9...|[-0.0017148339824...|(38770,[0,1,6,7,9...|
|i realli , really...|[realli, really, ...|(38670,[2,12,20,3...|[-0.0345313489987...|(38770,[2,12,20,3...|
|my good decent. t...|[good, decent, st...|(38670,[2,4,14,62...|[-0.0429038314947...|(38770,[2,4,14,62...|
|i shock star prev...|[shock, star, pre...|(38670,[2,16,19,2...|[0.02010906861687...|(38770,[2,16,19,2...|
|best friend i wen...|[best, friend, 

In [0]:
reviews_by_business_trf_df.printSchema()


root
 |-- business_id: string (nullable = true)
 |-- cleantext: string (nullable = true)
 |-- token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nostopwrd: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rawFeature: vector (nullable = true)
 |-- idf_vec: vector (nullable = true)
 |-- word_vec: vector (nullable = true)
 |-- comb_vec: vector (nullable = true)



In [0]:
def CosineSim(vec1, vec2):
    return np.dot(vec1, vec2) / np.sqrt(np.dot(vec1, vec1)) / np.sqrt(np.dot(vec2, vec2))

In [0]:
all_business_vecs = reviews_by_business_trf_df.select('business_id', 'word_vec').rdd.map(lambda x: (x[0], x[1])).collect()

In [0]:
all_business_vecs[1]

('--KQsXc-clkO7oHRqGzSzg',
 DenseVector([-0.0354, 0.0155, 0.0873, -0.0102, -0.0194, 0.0311, 0.0193, -0.0353, 0.0415, 0.0622, -0.0578, -0.0186, -0.0179, -0.0351, -0.0223, -0.021, 0.0047, 0.0781, -0.0122, 0.089, -0.0009, 0.005, 0.0012, 0.0182, -0.0229, 0.0225, -0.0586, 0.0194, 0.0113, 0.0439, -0.0353, -0.0265, -0.0417, -0.0068, -0.0099, -0.0575, -0.0094, 0.0231, 0.0243, -0.0132, -0.0122, -0.0692, 0.0101, -0.0009, -0.0241, 0.028, -0.0555, -0.0489, 0.0356, 0.0352, -0.0021, -0.0174, 0.0023, -0.0387, -0.0264, 0.0262, -0.0031, 0.0401, 0.0222, -0.0531, -0.0588, -0.0092, -0.019, 0.0073, 0.0434, -0.0124, 0.0042, -0.0601, -0.0114, 0.0071, 0.0302, 0.0332, -0.0151, -0.0099, -0.0154, 0.0272, 0.022, -0.0765, -0.066, 0.0331, -0.0255, -0.0251, -0.0412, -0.028, -0.0124, -0.0714, 0.0351, 0.0235, 0.0182, 0.0574, 0.0066, 0.0434, -0.0145, -0.0035, 0.028, 0.0041, 0.0423, 0.0316, -0.0098, 0.1174]))

In [0]:
reviews_by_user_df = spark.createDataFrame(user_concatreviews)

regexTokenizer = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'cleantext', outputCol = 'token')
stopWordsRemover = StopWordsRemover(inputCol = 'token', outputCol = 'nostopwrd')
countVectorizer = CountVectorizer(inputCol="nostopwrd", outputCol="rawFeature")
iDF = IDF(inputCol="rawFeature", outputCol="idf_vec")
word2Vec = Word2Vec(vectorSize = 100, minCount = 5, inputCol = 'nostopwrd', outputCol = 'word_vec', seed=123)
vectorAssembler = VectorAssembler(inputCols=['idf_vec', 'word_vec'], outputCol='comb_vec')
pipeline = Pipeline(stages=[regexTokenizer, stopWordsRemover, countVectorizer, iDF, word2Vec, vectorAssembler])

# fit the model
pipeline_mdl = pipeline.fit(reviews_by_user_df)

#save the pipeline model
pipeline_mdl.write().overwrite().save('pipe_txt')


pipeline_mdl = PipelineModel.load('pipe_txt')

reviews_by_user_trf_df = pipeline_mdl.transform(reviews_by_user_df)

reviews_by_user_trf_df.select( 'cleantext', 'nostopwrd', 'idf_vec', 'word_vec', 'comb_vec').show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|           cleantext|           nostopwrd|             idf_vec|            word_vec|            comb_vec|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|they host deliv h...|[host, deliv, hap...|(38670,[6,12,24,5...|[-0.0324616729200...|(38770,[6,12,24,5...|
|when buy coupl dr...|[buy, coupl, drin...|(38670,[5,8,13,17...|[-0.0655577084917...|(38770,[5,8,13,17...|
|unser eher besche...|[unser, eher, bes...|(38670,[25,28,133...|[0.21976470758543...|(38770,[25,28,133...|
|i love salon. the...|[love, salon, sta...|(38670,[16,32,116...|[-0.0284940421581...|(38770,[16,32,116...|
|ty fun best pad t...|[ty, fun, best, p...|(38670,[2,15,28,6...|[0.06743504623882...|(38770,[2,15,28,6...|
|i would like than...|[like, thank, mis...|(38670,[2,5,38,10...|[-0.0649818316675...|(38770,[2,5,38,10...|
|scrumptious!! the...|[scrumptious, p

In [0]:
reviews_by_user_trf_df.select( 'user_id','cleantext', 'idf_vec', 'word_vec', 'comb_vec').show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|             user_id|           cleantext|             idf_vec|            word_vec|            comb_vec|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|---1lKK3aKOuomHnw...|they host deliv h...|(38670,[6,12,24,5...|[-0.0324616729200...|(38770,[6,12,24,5...|
|--104qdWvE99vaoIs...|when buy coupl dr...|(38670,[5,8,13,17...|[-0.0655577084917...|(38770,[5,8,13,17...|
|--2vR0DIsmQ6WfcSz...|unser eher besche...|(38670,[25,28,133...|[0.21976470758543...|(38770,[25,28,133...|
|--B1_68kNRRoT9k61...|i love salon. the...|(38670,[16,32,116...|[-0.0284940421581...|(38770,[16,32,116...|
|-0S_XaK3Q_Mesal2U...|ty fun best pad t...|(38670,[2,15,28,6...|[0.06743504623882...|(38770,[2,15,28,6...|
|-1BYI0ft5KWoXQ_5u...|i would like than...|(38670,[2,5,38,10...|[-0.0649818316675...|(38770,[2,5,38,10...|
|-1Grce7VmCft_uq79...|scrumptious!! t

In [0]:
all_users_vecs = reviews_by_user_trf_df.select('user_id', 'word_vec').rdd.map(lambda x: (x[0], x[1])).collect()


In [0]:
all_users_vecs[0][0]

'---1lKK3aKOuomHnwAkAow'

In [0]:
all_users_vecs[0][1]

DenseVector([-0.0325, -0.0192, 0.068, -0.0534, -0.083, 0.0324, -0.075, 0.0285, 0.089, 0.011, -0.0468, 0.001, 0.0152, 0.0246, 0.0218, 0.0109, 0.0612, -0.027, -0.0209, 0.1093, 0.0089, 0.0542, 0.0087, -0.0004, 0.0586, 0.0243, -0.0697, 0.1032, 0.0357, -0.0888, -0.0827, 0.0486, 0.0428, -0.0338, -0.0546, -0.015, 0.0757, 0.0553, -0.0, 0.0272, 0.0869, -0.0642, 0.0764, -0.0347, -0.0404, 0.0903, -0.0744, 0.0494, -0.007, -0.0474, 0.0231, 0.0591, -0.0421, -0.0533, 0.0722, 0.0065, -0.0342, 0.1038, 0.0118, 0.0043, -0.0154, -0.0016, -0.1196, 0.0088, -0.0832, -0.0449, -0.018, -0.0063, 0.0887, -0.0247, 0.0395, 0.0363, -0.0363, 0.0269, -0.0541, 0.0314, 0.0333, -0.0092, -0.0383, 0.0226, 0.0211, 0.0615, -0.0601, -0.0184, -0.0118, -0.0909, 0.0643, -0.0002, 0.0884, 0.0349, -0.0862, 0.053, 0.0338, -0.0113, 0.0298, 0.0762, -0.0577, 0.0579, -0.0581, -0.0404])

In [0]:
for i in range(len(all_business_vecs)):
  CosineSim(all_business_vecs[i][1], all_users_vecs[0][1])